<a href="https://colab.research.google.com/github/DevDave96/AI-ML-Machine-Learning/blob/main/HEALTH_AND_FITNESS_PREDICTOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***HEALTH AND FITNESS PREDICTOR***

# ***IMPORT lIBRARIES***

In [2]:
import numpy as np
import matplotlib as pyplot
import pandas as pd
import seaborn as sns

# ***DATA PREPROCESSING***

In [3]:
Insurance = pd.read_csv('insurance.csv')
Obesity   = pd.read_csv('ObesityDataSet_raw_and_data_sinthetic.csv')

In [4]:
print(f"Insurance data: {Insurance.shape}")

Insurance data: (1338, 7)


In [5]:
print(f"Obesity data: {Obesity.shape}")

Obesity data: (2111, 17)


## ***REMOVE DUPLICATES***

In [6]:
Insurance = Insurance.drop_duplicates()
Obesity = Obesity.drop_duplicates()

# ***Separate the "Answer" (Target) from the "Data" (Features)*** ***& CONVERT WORDS TO NUMBERS*** ***FOR BOTH FEATURE AND TARGET***

In [7]:
X_reg = Insurance.drop('charges', axis=1)
y_reg = Insurance['charges']
X_reg = pd.get_dummies(X_reg, drop_first= True)


X_clf = Obesity.drop('NObeyesdad', axis=1)
y_clf = Obesity['NObeyesdad']
X_clf = pd.get_dummies(X_clf, drop_first= True)

## ***For the Target (What you are predicting)***\

In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_clf = le.fit_transform(y_clf)

# ***Splitting the Dataset into Training set and Test set***

In [9]:
from sklearn.model_selection import train_test_split
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)
X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(X_clf, y_clf, test_size=0.2, random_state=42)

In [10]:
print(f"Insurance Training rows: {len(X_train_reg)}")

Insurance Training rows: 1069


In [11]:
print(f"Obesity Training rows: {len(X_train_clf)}")

Obesity Training rows: 1669


# ***FEATURE SCALING***

In [12]:
from sklearn.preprocessing import StandardScaler
Scaler_reg = StandardScaler()
Scaler_clf = StandardScaler()

X_train_reg = Scaler_reg.fit_transform(X_train_reg)
X_test_reg = Scaler_reg.transform(X_test_reg)

X_train_clf = Scaler_clf.fit_transform(X_train_clf)
X_test_clf = Scaler_clf.transform(X_test_clf)

In [13]:
print(X_train_clf[0])

[-0.52747999  1.14792441 -0.27410858 -2.69748185  0.40484034 -0.01348087
  1.15273428 -1.09030512  0.98159534 -2.17157856 -2.7493589  -0.35628816
 -2.33858766  7.94935895 -0.139814   -0.22437943 -0.18804202 -1.41484931
  1.53342937 -0.05481613 -0.06940011  0.57988707 -0.16262   ]


# ***TIME TO GIVE THE AI ITS BRAIN*** ***TRAINING THE MODEL & WE WILL TRAIN TWO DIFFERENT MODELS***

In [14]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

reg_model = RandomForestRegressor(n_estimators=100, random_state=42)
reg_model.fit(X_train_reg, y_train_reg)

y_pred_reg = reg_model.predict(X_test_reg)
mae = mean_absolute_error(y_test_reg, y_pred_reg)
r2 = r2_score(y_test_reg, y_pred_reg)

In [15]:
print("--- 🏥 Insurance Predictor Performance ---")
print(f"Accuracy Grade (R2): {r2:.2%}")
print(f"Typical Error (MAE): ${mae:,.2f}")

if r2 > 0.8:
    print("\nResult: High Accuracy! This model is ready for your dashboard.")
else:
    print("\nResult: Moderate Accuracy. It can predict trends, but might miss specifics.")

--- 🏥 Insurance Predictor Performance ---
Accuracy Grade (R2): 87.82%
Typical Error (MAE): $2,663.33

Result: High Accuracy! This model is ready for your dashboard.


In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

clf_model = RandomForestClassifier(n_estimators = 100, random_state = 42)
clf_model.fit(X_train_clf, y_train_clf)

y_pred_clf = clf_model.predict(X_test_clf)

clf_accuracy = accuracy_score(y_test_clf, y_pred_clf)


In [17]:
print("--- ⚖️ Obesity Risk Classifier Performance ---")
print(f"Overall Accuracy: {clf_accuracy:.2%}")

--- ⚖️ Obesity Risk Classifier Performance ---
Overall Accuracy: 95.45%


In [18]:
print("\nDetailed Report:")
print(classification_report(y_test_clf, y_pred_clf))


Detailed Report:
              precision    recall  f1-score   support

           0       1.00      0.90      0.95        59
           1       0.81      0.95      0.87        61
           2       1.00      0.97      0.99        70
           3       1.00      1.00      1.00        64
           4       1.00      1.00      1.00        60
           5       0.94      0.89      0.92        55
           6       0.96      0.96      0.96        49

    accuracy                           0.95       418
   macro avg       0.96      0.95      0.95       418
weighted avg       0.96      0.95      0.96       418



# ***END***

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
import joblib
import os

path = '/content/drive/MyDrive/Health_AI_Project/'
if not os.path.exists(path):
    os.makedirs(path)


joblib.dump(reg_model, f'{path}insurance_model.pkl')
joblib.dump(clf_model, f'{path}obesity_model.pkl')

joblib.dump(Scaler_reg, f'{path}insurance_scaler.pkl')
joblib.dump(Scaler_clf, f'{path}obesity_scaler.pkl')

['/content/drive/MyDrive/Health_AI_Project/obesity_scaler.pkl']

In [21]:
print(f"✅ Success! All 4 components saved to: {path}")

✅ Success! All 4 components saved to: /content/drive/MyDrive/Health_AI_Project/


## ***SIMPLE PREDICTION WHERE I TYPE MY AGE AND WEIGHT AND SEE WHAT THE AI PREDICT***

In [22]:
def predict_my_obesity_risk(user_input_dict):
  input_df = pd.DataFrame([user_input_dict])
  input_encoded = pd.get_dummies(input_df)
  for col in X_clf.columns:
        if col not in input_encoded.columns:
            input_encoded[col] = 0

  input_encoded = input_encoded[X_clf.columns]
  input_scaled = Scaler_clf.transform(input_encoded)
  prediction = clf_model.predict(input_scaled)
  return prediction[0]

my_data = {
    'Age': 25,
    'Height': 1.88,
    'Weight': 95,
    'family_history_with_overweight': 'no',
    'FAVC': 'no',   # Frequent consumption of high caloric food
    'FCVC': 3,      # Vegetables in meals
    'NCP': 2,       # Number of main meals
    'CAEC': 'Sometimes',
    'SMOKE': 'no',
    'CH2O': 2,      # Water intake
    'SCC': 'yes',    # Calories monitoring
    'FAF': 1,       # Physical activity
    'TUE': 1,       # Time using electronics
    'CALC': 'Sometimes',
    'MTRANS': 'Automobile'
}
result = predict_my_obesity_risk(my_data)

In [23]:
print(f"--- 🤖 AI DIAGNOSIS ---")
print(f"The AI predicts your category is: {result}")

--- 🤖 AI DIAGNOSIS ---
The AI predicts your category is: 6


In [24]:
result_num = predict_my_obesity_risk(my_data)

In [25]:
print(f"--- 🤖 AI DIAGNOSIS ---")
print(f"Numeric Output: {result_num}")

--- 🤖 AI DIAGNOSIS ---
Numeric Output: 6


In [26]:
res_num = predict_my_obesity_risk(my_data)

labels = [
    'Insufficient_Weight',
    'Normal_Weight',
    'Obesity_Type_I',
    'Obesity_Type_II',
    'Obesity_Type_III',
    'Overweight_Level_I',
    'Overweight_Level_II'
]



In [27]:
print(f"--- 🤖 AI DIAGNOSIS ---")
if res_num < len(labels):
    print(f"The AI classifies you as: {labels[res_num]}")
else:
    print(f"The AI gave a code ({res_num}) that isn't in the list.")

--- 🤖 AI DIAGNOSIS ---
The AI classifies you as: Overweight_Level_II


# New Section

In [31]:
# Run this in a new cell in Colab
import pandas as pd
# Load your obesity dataset
df_obesity = pd.read_csv('ObesityDataSet_raw_and_data_sinthetic.csv')
# Run the same dummy/prep you did for training
df_ob_dummies = pd.get_dummies(df_obesity.drop('NObeyesdad', axis=1), drop_first=True)


In [32]:
print(df_ob_dummies.columns.tolist())

['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE', 'Gender_Male', 'family_history_with_overweight_yes', 'FAVC_yes', 'CAEC_Frequently', 'CAEC_Sometimes', 'CAEC_no', 'SMOKE_yes', 'SCC_yes', 'CALC_Frequently', 'CALC_Sometimes', 'CALC_no', 'MTRANS_Bike', 'MTRANS_Motorbike', 'MTRANS_Public_Transportation', 'MTRANS_Walking']
